# Aula 2: Introdução ao Python - Scraping (Cont.)

Em nossa última aula aprendemos como realizar uma requisição para uma página afim de obter o seu código fonte e, assim, realizar alguma coleta de dados. Mais especificamente, aprendemos a realizar o método *GET* a partir do pacote *requests*

Hoje aprenderemos como realizar o método *POST*. Utilizaremos o método *POST* sempre que for necessário enviar informações específicar para a construção do código fonte da página.

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import time
import json
import csv
import random

Imagine que queiramos obter dados de empresas em PE. Nos dirigimos ao site da [JUCEPE](https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx) [https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx]. Pode-se observar que no site há um formulário a ser preenchido. Podemos pesquisar por CPF ou CNPJ, nome, etc...

Observe que devemos informar ao site alguns parâmetros para a pesquisa e, a partir dessas informações, o site irá construir uma outra página com as informações condizentes com nossos parâmetros. Nos referimos a este tipo de pesquisa como um *POST* já que estamos postando informações necessárias para o website construir a página.

Vamos realizar um *GET* para a página de pesquisa inicial e analisar o resultado.

In [30]:
resultado = requests.get("https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx")
t = resultado.text
print(BeautifulSoup(t, 'lxml').prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
 <head>
  <title>
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Consulta Empresa" property="og:title"/>
  <meta content="Busca Empresa" name="description"/>
  <meta content="noindex, nofollow" name="robots"/>
  <script type="text/javascript">
   var GB_ROOT_DIR = "./greybox/";
  </script>
  <!-- Folha de Estilo do Portal -->
  <link href="css/estilo.css" rel="stylesheet" type="text/css"/>
  <link href="css/start/jquery-ui-1.8.2.custom.css" rel="stylesheet" type="text/css"/>
  <link href="imagens/favicon.ico" rel="shortcut icon"/>
  <link href="greybox/gb_styles.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="css/datepicker.css" rel="stylesheet" type="text/css"/>
  <link href="js/datatable/med

Podemos observar que o site contém um formulário (&lt;form>) a ser preenchido. Vejamos seu código:
    
<img src="1.png" width="800" height="200">

Vamos preencher o formulário e analisar o tráfego da web para ver o que acontece.

<img src="2.png" width="800" height="200">

Como podemos observar, o site enviou os parâmetros que preenchemos (junto com alguns outros) através do método *POST* de forma a construir a página com as informações desejadas. Os seguintes parâmetros foram enviados:

<img src="3.png" width="800" height="200">

Ou seja, se quisermos pegar as informações ná página que foi gerada após preenchermos o formulário, teremos que preencher o formulário com o python, passando os parâmetros na requisição *POST*.

In [29]:
cpf = "002.301.204/82"
url = "https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx"

Para definir um set de parâmetros a serem passados em um request *POST* utilizaremos uma lista e, dentro da lista, cada elemento será uma *Tuple*, cada uma contendo o nome do parâmetro e o seu respectivo valor. 

In [5]:
parametros = [('__EVENTTARGET', ''), 
              ('__EVENTARGUMENT', ''),
              ('__VIEWSTATE', ''),
              ('__VIEWSTATEGENERATOR', ''),
              ('__EVENTVALIDATION', ''),
              ('txtNire', ''),
              ('txtCNPJ', ''),
              ('txtNomePessoa', ''),
              ('txtNomeSocio', ''),
              ('txtCPFSocio', cpf),
              ('txtBairroRapida', ''),
              ('txtLogradouroRapida', ''),
              ('btnPesquisaRapida', 'Pesquisar'),
             ]

In [6]:
parametros

[('__EVENTTARGET', ''),
 ('__EVENTARGUMENT', ''),
 ('__VIEWSTATE', ''),
 ('__VIEWSTATEGENERATOR', ''),
 ('__EVENTVALIDATION', ''),
 ('txtNire', ''),
 ('txtCNPJ', ''),
 ('txtNomePessoa', ''),
 ('txtNomeSocio', ''),
 ('txtCPFSocio', '002.301.204/82'),
 ('txtBairroRapida', ''),
 ('txtLogradouroRapida', ''),
 ('btnPesquisaRapida', 'Pesquisar')]

E os parâmetros: EVENTTARGET, EVENTARGUMENT, VIEWSTATE, VIEWSTATEGENERATOR e EVENTVALIDATION?

Esses são parâmetros de uma requisição que utiliza AJAX. Eles serão encontrados na página principal do site

In [31]:
soup_main = BeautifulSoup(t, 'lxml')

In [32]:
soup_main.findAll('input', attrs={'name' : '__VIEWSTATE'})

[<input id="__VIEWSTATE" name="__VIEWSTATE" type="hidden" value="/wEPDwUJNjU3NzAxMTkxD2QWAgIDD2QWAgIBD2QWBgIHDzwrAA0BDBQrAANkFCsAAhYCHghTZWxlY3RlZGdkZGQCCQ9kFgICAQ8PFgIeBFRleHRlZGQCCw9kFgRmD2QWAgIDDw8WAh4HVmlzaWJsZWhkFg4CAQ8QZGQWAWZkAgkPEGQPFroBZgIBAgICAwIEAgUCBgIHAggCCQIKAgsCDAINAg4CDwIQAhECEgITAhQCFQIWAhcCGAIZAhoCGwIcAh0CHgIfAiACIQIiAiMCJAIlAiYCJwIoAikCKgIrAiwCLQIuAi8CMAIxAjICMwI0AjUCNgI3AjgCOQI6AjsCPAI9Aj4CPwJAAkECQgJDAkQCRQJGAkcCSAJJAkoCSwJMAk0CTgJPAlACUQJSAlMCVAJVAlYCVwJYAlkCWgJbAlwCXQJeAl8CYAJhAmICYwJkAmUCZgJnAmgCaQJqAmsCbAJtAm4CbwJwAnECcgJzAnQCdQJ2AncCeAJ5AnoCewJ8An0CfgJ/AoABAoEBAoIBAoMBAoQBAoUBAoYBAocBAogBAokBAooBAosBAowBAo0BAo4BAo8BApABApEBApIBApMBApQBApUBApYBApcBApgBApkBApoBApsBApwBAp0BAp4BAp8BAqABAqEBAqIBAqMBAqQBAqUBAqYBAqcBAqgBAqkBAqoBAqsBAqwBAq0BAq4BAq8BArABArEBArIBArMBArQBArUBArYBArcBArgBArkBFroBEGVlZxAFDEFCUkVVIEUgTElNQQUENTEzNWcQBRVBRk9HQURPUyBEQSBJTkdBWkVJUkEFBDUxMzZnEAUIQUZSw4JOSU8FBDUxMzdnEAUJQUdSRVNUSU5BBQQ1MTM4ZxAFC8OBR1VBIFBSRVRBBQQ1MTQwZxAFDMOBR1VBUyBCRUxBUwUENTE

In [33]:
result = soup_main.findAll('input', attrs={'name' : '__VIEWSTATE'})[0]

In [34]:
VIEWSTATE =  result['value'];

In [35]:
VIEWSTATE

'/wEPDwUJNjU3NzAxMTkxD2QWAgIDD2QWAgIBD2QWBgIHDzwrAA0BDBQrAANkFCsAAhYCHghTZWxlY3RlZGdkZGQCCQ9kFgICAQ8PFgIeBFRleHRlZGQCCw9kFgRmD2QWAgIDDw8WAh4HVmlzaWJsZWhkFg4CAQ8QZGQWAWZkAgkPEGQPFroBZgIBAgICAwIEAgUCBgIHAggCCQIKAgsCDAINAg4CDwIQAhECEgITAhQCFQIWAhcCGAIZAhoCGwIcAh0CHgIfAiACIQIiAiMCJAIlAiYCJwIoAikCKgIrAiwCLQIuAi8CMAIxAjICMwI0AjUCNgI3AjgCOQI6AjsCPAI9Aj4CPwJAAkECQgJDAkQCRQJGAkcCSAJJAkoCSwJMAk0CTgJPAlACUQJSAlMCVAJVAlYCVwJYAlkCWgJbAlwCXQJeAl8CYAJhAmICYwJkAmUCZgJnAmgCaQJqAmsCbAJtAm4CbwJwAnECcgJzAnQCdQJ2AncCeAJ5AnoCewJ8An0CfgJ/AoABAoEBAoIBAoMBAoQBAoUBAoYBAocBAogBAokBAooBAosBAowBAo0BAo4BAo8BApABApEBApIBApMBApQBApUBApYBApcBApgBApkBApoBApsBApwBAp0BAp4BAp8BAqABAqEBAqIBAqMBAqQBAqUBAqYBAqcBAqgBAqkBAqoBAqsBAqwBAq0BAq4BAq8BArABArEBArIBArMBArQBArUBArYBArcBArgBArkBFroBEGVlZxAFDEFCUkVVIEUgTElNQQUENTEzNWcQBRVBRk9HQURPUyBEQSBJTkdBWkVJUkEFBDUxMzZnEAUIQUZSw4JOSU8FBDUxMzdnEAUJQUdSRVNUSU5BBQQ1MTM4ZxAFC8OBR1VBIFBSRVRBBQQ1MTQwZxAFDMOBR1VBUyBCRUxBUwUENTE0MWcQBQlBTEFHT0lOSEEFBDUxNDNnEAUIQUxJQU7Dh0EFBDUxNDZnEAUHQUxUSU5

In [36]:
result = soup_main.findAll('input', attrs={'name' : '__VIEWSTATEGENERATOR'})[0]
VIEWSTATEGENERATOR =  result['value']

In [37]:
VIEWSTATEGENERATOR

'507AACDF'

In [38]:
result = soup_main.findAll('input', attrs={'name' : '__EVENTVALIDATION'})[0]
EVENTVALIDATION =  result['value']

In [39]:
EVENTVALIDATION

'/wEWCwKesMiVCwLqmbXUAQK8hKiECwL2kJ7aAwKU1dWlBQKa3PL4DQKnk67+CgKW/MmsDgKD7bLYAwKvjZz3AwK8msX8DCIRGBssSJ6KRznuDaXfKr4bGJDE'

In [40]:
parametros = [('__EVENTTARGET', ''), 
              ('__EVENTARGUMENT', ''),
              ('__VIEWSTATE', VIEWSTATE),
              ('__VIEWSTATEGENERATOR', VIEWSTATEGENERATOR),
              ('__EVENTVALIDATION', EVENTVALIDATION),
              ('txtNire', ''),
              ('txtCNPJ', ''),
              ('txtNomePessoa', ''),
              ('txtNomeSocio', ''),
              ('txtCPFSocio', cpf),
              ('txtBairroRapida', ''),
              ('txtLogradouroRapida', ''),
              ('btnPesquisaRapida', 'Pesquisar'),
             ]

In [41]:
parametros

[('__EVENTTARGET', ''),
 ('__EVENTARGUMENT', ''),
 ('__VIEWSTATE',
  '/wEPDwUJNjU3NzAxMTkxD2QWAgIDD2QWAgIBD2QWBgIHDzwrAA0BDBQrAANkFCsAAhYCHghTZWxlY3RlZGdkZGQCCQ9kFgICAQ8PFgIeBFRleHRlZGQCCw9kFgRmD2QWAgIDDw8WAh4HVmlzaWJsZWhkFg4CAQ8QZGQWAWZkAgkPEGQPFroBZgIBAgICAwIEAgUCBgIHAggCCQIKAgsCDAINAg4CDwIQAhECEgITAhQCFQIWAhcCGAIZAhoCGwIcAh0CHgIfAiACIQIiAiMCJAIlAiYCJwIoAikCKgIrAiwCLQIuAi8CMAIxAjICMwI0AjUCNgI3AjgCOQI6AjsCPAI9Aj4CPwJAAkECQgJDAkQCRQJGAkcCSAJJAkoCSwJMAk0CTgJPAlACUQJSAlMCVAJVAlYCVwJYAlkCWgJbAlwCXQJeAl8CYAJhAmICYwJkAmUCZgJnAmgCaQJqAmsCbAJtAm4CbwJwAnECcgJzAnQCdQJ2AncCeAJ5AnoCewJ8An0CfgJ/AoABAoEBAoIBAoMBAoQBAoUBAoYBAocBAogBAokBAooBAosBAowBAo0BAo4BAo8BApABApEBApIBApMBApQBApUBApYBApcBApgBApkBApoBApsBApwBAp0BAp4BAp8BAqABAqEBAqIBAqMBAqQBAqUBAqYBAqcBAqgBAqkBAqoBAqsBAqwBAq0BAq4BAq8BArABArEBArIBArMBArQBArUBArYBArcBArgBArkBFroBEGVlZxAFDEFCUkVVIEUgTElNQQUENTEzNWcQBRVBRk9HQURPUyBEQSBJTkdBWkVJUkEFBDUxMzZnEAUIQUZSw4JOSU8FBDUxMzdnEAUJQUdSRVNUSU5BBQQ1MTM4ZxAFC8OBR1VBIFBSRVRBBQQ1MTQwZxAFDMOBR1VBUyBCRUxBUw

Uma *Tuple* é quase igual a uma lista. A única diferença é que listas são mutaveis e uma *Tuple* é imutável.

Podemos, agora, efetuar a requisição utlizando o método *POST*, passando a lista *data* como parâmetro.

In [50]:
dados_jucepe = requests.post(url, data=parametros)

In [51]:
dados_jucepe

<Response [200]>

In [52]:
soup = BeautifulSoup(dados_jucepe.content, 'lxml')

In [53]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head><title>
</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="Consulta Empresa" property="og:title"/><meta content="Busca Empresa" name="description"/><meta content="noindex, nofollow" name="robots"/>
<script type="text/javascript">
		var GB_ROOT_DIR = "./greybox/";
	</script>
<!-- Folha de Estilo do Portal -->
<link href="css/estilo.css" rel="stylesheet" type="text/css"/><link href="css/start/jquery-ui-1.8.2.custom.css" rel="stylesheet" type="text/css"/><link href="imagens/favicon.ico" rel="shortcut icon"/><link href="greybox/gb_styles.css" media="all" rel="stylesheet" type="text/css"/><link href="css/datepicker.css" rel="stylesheet" type="text/css"/><link href="js/datatable/media/css/demo_page.css" rel="stylesheet" type="text/

In [60]:
headers_parameters = {
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language":"en-US,en;q=0.9",
    "Cache-Control":"max-age=0",
    "Content-Length":"17927",
    "Content-Type":"application/x-www-form-urlencoded",
    "Origin":"https://iged.jucepe.pe.gov.br",
    "Referer":"https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx",
    "Sec-Ch-Ua":"'Chromium';v='116', 'Not)A;Brand';v='24', 'Google Chrome';v='116'",
    "Sec-Ch-Ua-Mobile":"?0",
    "Sec-Ch-Ua-Platform":"'macOS'",
    "Sec-Fetch-Dest":"document",
    "Sec-Fetch-Mode":"navigate",
    "Sec-Fetch-Site":"same-origin",
    "Sec-Fetch-User":"?1",
    "Upgrade-Insecure-Requests":"1",
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
    }

In [61]:
headers_parameters

{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
 'Accept-Encoding': 'gzip, deflate, br',
 'Accept-Language': 'en-US,en;q=0.9',
 'Cache-Control': 'max-age=0',
 'Content-Length': '17927',
 'Content-Type': 'application/x-www-form-urlencoded',
 'Origin': 'https://iged.jucepe.pe.gov.br',
 'Referer': 'https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx',
 'Sec-Ch-Ua': "'Chromium';v='116', 'Not)A;Brand';v='24', 'Google Chrome';v='116'",
 'Sec-Ch-Ua-Mobile': '?0',
 'Sec-Ch-Ua-Platform': "'macOS'",
 'Sec-Fetch-Dest': 'document',
 'Sec-Fetch-Mode': 'navigate',
 'Sec-Fetch-Site': 'same-origin',
 'Sec-Fetch-User': '?1',
 'Upgrade-Insecure-Requests': '1',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

In [56]:
dados_jucepe = requests.post(url, data=parametros, headers=headers_parameters)

In [57]:
dados_jucepe

<Response [200]>

In [58]:
soup = BeautifulSoup(dados_jucepe.content, 'lxml')

In [59]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head><title>
</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="Consulta Empresa" property="og:title"/><meta content="Busca Empresa" name="description"/><meta content="noindex, nofollow" name="robots"/>
<script type="text/javascript">
		var GB_ROOT_DIR = "./greybox/";
	</script>
<!-- Folha de Estilo do Portal -->
<link href="css/estilo.css" rel="stylesheet" type="text/css"/><link href="css/start/jquery-ui-1.8.2.custom.css" rel="stylesheet" type="text/css"/><link href="imagens/favicon.ico" rel="shortcut icon"/><link href="greybox/gb_styles.css" media="all" rel="stylesheet" type="text/css"/><link href="css/datepicker.css" rel="stylesheet" type="text/css"/><link href="js/datatable/media/css/demo_page.css" rel="stylesheet" type="text/

O que tentamos não funcionou por que o site usa cookeis para validar a sessão. Precisamos utilizar a mesma sessão no requests para realizar a requisição.

Construiremos, então, uma sessão a ser reutilizada a cada request.

In [63]:
s = requests.Session()

In [64]:
s

In [65]:
resultado = s.get("https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx")
t = resultado.text

In [66]:
soup_main = BeautifulSoup(t, 'lxml')
result = soup_main.findAll('input', attrs={'name' : '__VIEWSTATE'})[0]
VIEWSTATE =  result['value'];
result = soup_main.findAll('input', attrs={'name' : '__VIEWSTATEGENERATOR'})[0]
VIEWSTATEGENERATOR =  result['value']
result = soup_main.findAll('input', attrs={'name' : '__EVENTVALIDATION'})[0]
EVENTVALIDATION =  result['value']

parametros = [('__EVENTTARGET', ''), 
              ('__EVENTARGUMENT', ''),
              ('__VIEWSTATE', VIEWSTATE),
              ('__VIEWSTATEGENERATOR', VIEWSTATEGENERATOR),
              ('__EVENTVALIDATION', EVENTVALIDATION),
              ('txtNire', ''),
              ('txtCNPJ', ''),
              ('txtNomePessoa', ''),
              ('txtNomeSocio', ''),
              ('txtCPFSocio', cpf),
              ('txtBairroRapida', ''),
              ('txtLogradouroRapida', ''),
              ('btnPesquisaRapida', 'Pesquisar'),
             ]

In [67]:
parametros

[('__EVENTTARGET', ''),
 ('__EVENTARGUMENT', ''),
 ('__VIEWSTATE',
  '/wEPDwUJNjU3NzAxMTkxD2QWAgIDD2QWAgIBD2QWBgIHDzwrAA0BDBQrAANkFCsAAhYCHghTZWxlY3RlZGdkZGQCCQ9kFgICAQ8PFgIeBFRleHRlZGQCCw9kFgRmD2QWAgIDDw8WAh4HVmlzaWJsZWhkFg4CAQ8QZGQWAWZkAgkPEGQPFroBZgIBAgICAwIEAgUCBgIHAggCCQIKAgsCDAINAg4CDwIQAhECEgITAhQCFQIWAhcCGAIZAhoCGwIcAh0CHgIfAiACIQIiAiMCJAIlAiYCJwIoAikCKgIrAiwCLQIuAi8CMAIxAjICMwI0AjUCNgI3AjgCOQI6AjsCPAI9Aj4CPwJAAkECQgJDAkQCRQJGAkcCSAJJAkoCSwJMAk0CTgJPAlACUQJSAlMCVAJVAlYCVwJYAlkCWgJbAlwCXQJeAl8CYAJhAmICYwJkAmUCZgJnAmgCaQJqAmsCbAJtAm4CbwJwAnECcgJzAnQCdQJ2AncCeAJ5AnoCewJ8An0CfgJ/AoABAoEBAoIBAoMBAoQBAoUBAoYBAocBAogBAokBAooBAosBAowBAo0BAo4BAo8BApABApEBApIBApMBApQBApUBApYBApcBApgBApkBApoBApsBApwBAp0BAp4BAp8BAqABAqEBAqIBAqMBAqQBAqUBAqYBAqcBAqgBAqkBAqoBAqsBAqwBAq0BAq4BAq8BArABArEBArIBArMBArQBArUBArYBArcBArgBArkBFroBEGVlZxAFDEFCUkVVIEUgTElNQQUENTEzNWcQBRVBRk9HQURPUyBEQSBJTkdBWkVJUkEFBDUxMzZnEAUIQUZSw4JOSU8FBDUxMzdnEAUJQUdSRVNUSU5BBQQ1MTM4ZxAFC8OBR1VBIFBSRVRBBQQ1MTQwZxAFDMOBR1VBUyBCRUxBUw

In [68]:
dados_jucepe = s.post(url, data=parametros, headers=headers_parameters)

In [69]:
soup = BeautifulSoup(dados_jucepe.content, 'lxml')

In [70]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head><title>
</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="Consulta Empresa" property="og:title"/><meta content="Busca Empresa" name="description"/><meta content="noindex, nofollow" name="robots"/>
<script type="text/javascript">
		var GB_ROOT_DIR = "./greybox/";
	</script>
<!-- Folha de Estilo do Portal -->
<link href="css/estilo.css" rel="stylesheet" type="text/css"/><link href="css/start/jquery-ui-1.8.2.custom.css" rel="stylesheet" type="text/css"/><link href="imagens/favicon.ico" rel="shortcut icon"/><link href="greybox/gb_styles.css" media="all" rel="stylesheet" type="text/css"/><link href="css/datepicker.css" rel="stylesheet" type="text/css"/><link href="js/datatable/media/css/demo_page.css" rel="stylesheet" type="text/

In [77]:
table_dados = soup.find("table", attrs={"id":"tblEmpresas"})

In [78]:
table_dados

<table border="0" cellpadding="0" cellspacing="0" class="display" id="tblEmpresas">
<thead>
<tr align="left">
<th>Nire               
</th>
<th>Empresa / LocalizaÃ§Ã£o       
</th>
<th>Natureza JurÃ­dica  
</th>
<th>CNPJ               
</th>
<th>SituaÃ§Ã£o           
</th>
<th>MunicÃ­pio          
</th>
</tr>
</thead>
<tr>
<td><a classe="greybox" href="RelBuscaEmpresa.aspx?NR_NIRE=26200575378" rel="gb_page_center[800, 500]" title="Clique aqui para ver os dados da empresa.">26200575378</a></td>
<td>
<!--
					  <a title="Clique aqui para ver o endereÃ§o da empresa no mapa." href="http://maps.google.com.br/maps?q=AVENIDA CAXANGA&hl=pt-BR&ie=UTF8&hnear=R.+C%C3%A2ndido+Lacerda,+3839+-+IPUTINGA,+RECIFE&gl=br&t=m&z=17classe="greybox" rel="gb_page_center[800, 500]"> AGRO FACTORING FOMENTO COMERCIAL LTDA </a>
					  -->
<a href="http://maps.google.com.br/maps?f=q&amp;source=s_q&amp;hl=pt-BR&amp;geocode=&amp;q=AVENIDA CAXANGA,3839,RECIFE" target="_blank" title="Clique aqui para ver o endereÃ§o 

In [79]:
for item in table_dados.find_all('tr'):
    print(item)

<tr align="left">
<th>Nire               
</th>
<th>Empresa / LocalizaÃ§Ã£o       
</th>
<th>Natureza JurÃ­dica  
</th>
<th>CNPJ               
</th>
<th>SituaÃ§Ã£o           
</th>
<th>MunicÃ­pio          
</th>
</tr>
<tr>
<td><a classe="greybox" href="RelBuscaEmpresa.aspx?NR_NIRE=26200575378" rel="gb_page_center[800, 500]" title="Clique aqui para ver os dados da empresa.">26200575378</a></td>
<td>
<!--
					  <a title="Clique aqui para ver o endereÃ§o da empresa no mapa." href="http://maps.google.com.br/maps?q=AVENIDA CAXANGA&hl=pt-BR&ie=UTF8&hnear=R.+C%C3%A2ndido+Lacerda,+3839+-+IPUTINGA,+RECIFE&gl=br&t=m&z=17classe="greybox" rel="gb_page_center[800, 500]"> AGRO FACTORING FOMENTO COMERCIAL LTDA </a>
					  -->
<a href="http://maps.google.com.br/maps?f=q&amp;source=s_q&amp;hl=pt-BR&amp;geocode=&amp;q=AVENIDA CAXANGA,3839,RECIFE" target="_blank" title="Clique aqui para ver o endereÃ§o da empresa no mapa."> AGRO FACTORING FOMENTO COMERCIAL LTDA </a>
</td>
<td>SOCIEDADE EMPRESÃRIA LIMI

In [80]:
item

<tr>
<td><a classe="greybox" href="RelBuscaEmpresa.aspx?NR_NIRE=26200371985" rel="gb_page_center[800, 500]" title="Clique aqui para ver os dados da empresa.">26200371985</a></td>
<td>
<!--
					  <a title="Clique aqui para ver o endereÃ§o da empresa no mapa." href="http://maps.google.com.br/maps?q=RUA CAPITAO LIMA&hl=pt-BR&ie=UTF8&hnear=R.+C%C3%A2ndido+Lacerda,+250+-+SANTO AMARO,+RECIFE&gl=br&t=m&z=17classe="greybox" rel="gb_page_center[800, 500]"> TV JORNAL DO COMERCIO LTDA </a>
					  -->
<a href="http://maps.google.com.br/maps?f=q&amp;source=s_q&amp;hl=pt-BR&amp;geocode=&amp;q=RUA CAPITAO LIMA,250,RECIFE" target="_blank" title="Clique aqui para ver o endereÃ§o da empresa no mapa."> TV JORNAL DO COMERCIO LTDA </a>
</td>
<td>SOCIEDADE EMPRESÃRIA LIMITADA</td>
<td>09.045.816.0001/06</td>
<td>EXTINTA</td>
<td>RECIFE</td>
</tr>

In [81]:
item.find_all("td")

[<td><a classe="greybox" href="RelBuscaEmpresa.aspx?NR_NIRE=26200371985" rel="gb_page_center[800, 500]" title="Clique aqui para ver os dados da empresa.">26200371985</a></td>,
 <td>
 <!--
 					  <a title="Clique aqui para ver o endereÃ§o da empresa no mapa." href="http://maps.google.com.br/maps?q=RUA CAPITAO LIMA&hl=pt-BR&ie=UTF8&hnear=R.+C%C3%A2ndido+Lacerda,+250+-+SANTO AMARO,+RECIFE&gl=br&t=m&z=17classe="greybox" rel="gb_page_center[800, 500]"> TV JORNAL DO COMERCIO LTDA </a>
 					  -->
 <a href="http://maps.google.com.br/maps?f=q&amp;source=s_q&amp;hl=pt-BR&amp;geocode=&amp;q=RUA CAPITAO LIMA,250,RECIFE" target="_blank" title="Clique aqui para ver o endereÃ§o da empresa no mapa."> TV JORNAL DO COMERCIO LTDA </a>
 </td>,
 <td>SOCIEDADE EMPRESÃRIA LIMITADA</td>,
 <td>09.045.816.0001/06</td>,
 <td>EXTINTA</td>,
 <td>RECIFE</td>]

In [82]:
for row in table_dados.find_all('tr'):
    cols = row.find_all('td')
    print(cols)

[]
[<td><a classe="greybox" href="RelBuscaEmpresa.aspx?NR_NIRE=26200575378" rel="gb_page_center[800, 500]" title="Clique aqui para ver os dados da empresa.">26200575378</a></td>, <td>
<!--
					  <a title="Clique aqui para ver o endereÃ§o da empresa no mapa." href="http://maps.google.com.br/maps?q=AVENIDA CAXANGA&hl=pt-BR&ie=UTF8&hnear=R.+C%C3%A2ndido+Lacerda,+3839+-+IPUTINGA,+RECIFE&gl=br&t=m&z=17classe="greybox" rel="gb_page_center[800, 500]"> AGRO FACTORING FOMENTO COMERCIAL LTDA </a>
					  -->
<a href="http://maps.google.com.br/maps?f=q&amp;source=s_q&amp;hl=pt-BR&amp;geocode=&amp;q=AVENIDA CAXANGA,3839,RECIFE" target="_blank" title="Clique aqui para ver o endereÃ§o da empresa no mapa."> AGRO FACTORING FOMENTO COMERCIAL LTDA </a>
</td>, <td>SOCIEDADE EMPRESÃRIA LIMITADA</td>, <td>24.407.504.0001/99</td>, <td>CANCELADA - ART.60 LEI 8934/94</td>, <td>RECIFE</td>]
[<td><a classe="greybox" href="RelBuscaEmpresa.aspx?NR_NIRE=26300003139" rel="gb_page_center[800, 500]" title="Clique aq

Podemos fazer um loop no objeto *table_dados* para salvar os campos em um *json*

In [83]:
fields = {}
for row in table_dados.find_all('tr'):
    cols = row.find_all('td')
    fields[cols[0].text] = cols[1].text

IndexError: list index out of range

### TRY, EXCEPT, ELSE

Utilizaremos *TRY, EXCEPT, ELSE* para captar exceções em nossos códigos

```
try:
   You do your operations here;
   ......................
except IndexError:
   If there is ExceptionI, then execute this block.
except ExceptionII:
   If there is ExceptionII, then execute this block.
   ......................
else:
   If there is no exception then execute this block. 
```

Vamos ver um exemplo em que abriremos um arquivo *.txt* para escrever algo nele.

Abriremos um arquivo em modo *write* inicialmente:

In [84]:
num = [-5,-4,-3,-2,-1,0,1,2,3,4,5,0]

In [85]:
num

[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 0]

In [86]:
for i in num:
    g = 10/i
    print(g)

-2.0
-2.5
-3.3333333333333335
-5.0
-10.0


ZeroDivisionError: division by zero

In [87]:
for i in num:
    try:
        g = 10/i
    except:
        continue
    else:
        print(g)

-2.0
-2.5
-3.3333333333333335
-5.0
-10.0
10.0
5.0
3.3333333333333335
2.5
2.0


Vamos implementar, então, exceções em nosso código de coleta de forma a capturar todas as informações das empresas.

In [88]:
fields = {}
for row in table_dados.find_all('tr'):
    cols = row.find_all('td')
    try:
        fields[cols[0].text] = cols[1].text
    except:
        continue
    else:
        fields[cols[0].text] = cols[1].text

In [89]:
fields

{'26200575378': '\n\n AGRO FACTORING FOMENTO COMERCIAL LTDA \n',
 '26300003139': '\n\n ALFANOR S/A AGRICULTURA INDUSTRIA E COMERCIO DO NORDESTE \n',
 '26202114211': '\n\n ARACAJU LOCADORA DE EQUIPAMENTOS PARA COGERAÃ\x87Ã\x83O LTDA \n',
 '26201156026': '\n\n AVICOLA DAGEMA LTDA \n',
 '26300002132': '\n\n AVICOLA DAGEMA S/A \n',
 '26200529422': '\n\n BOMPRECO COMERCIAL DE ALIMENTOS LTDA \n',
 '26300008173': '\n\n BOMPRECO PARTICIPACOES S/A \n',
 '26300009927': '\n\n BOMPRECO TRUST DE RECEBIVEIS S/A \n',
 '26200580916': '\n\n BOMSERV FACTORING FOMENTO COMERCIAL LTDA \n',
 '26300010160': '\n\n COMPANHIA BOMPRECO DE PARTICIPACOES E EMPREENDIMENTOS \n',
 '26202481711': '\n\n EDITORA JORNAL DO COMMERCIO LTDA \n',
 '26201086834': '\n\n ELO COMUNICAÃ\x87Ã\x83O LTDA \n',
 '26201223572': '\n\n EPL EMPREENDIMENTOS E PARTICIPACOES LTDA \n',
 '26300006545': '\n\n FRUTICULTURA PERIPERI S/A \n',
 '26202139877': '\n\n GUARARAPES PARKING LTDA. \n',
 '26201184763': '\n\n HIPERPETI & PETIPRECO COMERCIO I

In [90]:
cpf = "002.301.204/82"
url = "https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx"

dados_empresas = []

with requests.Session() as s:
    resultado = s.get(url)
    t = resultado.text
    soup_main = BeautifulSoup(t, 'lxml')
    result = soup_main.findAll('input', attrs={'name' : '__VIEWSTATE'})[0]
    VIEWSTATE =  result['value'];
    result = soup_main.findAll('input', attrs={'name' : '__VIEWSTATEGENERATOR'})[0]
    VIEWSTATEGENERATOR =  result['value']
    result = soup_main.findAll('input', attrs={'name' : '__EVENTVALIDATION'})[0]
    EVENTVALIDATION =  result['value']
    #
    parametros = [('__EVENTTARGET', ''), 
                  ('__EVENTARGUMENT', ''),
                  ('__VIEWSTATE', VIEWSTATE),
                  ('__VIEWSTATEGENERATOR', VIEWSTATEGENERATOR),
                  ('__EVENTVALIDATION', EVENTVALIDATION),
                  ('txtNire', ''),
                  ('txtCNPJ', ''),
                  ('txtNomePessoa', ''),
                  ('txtNomeSocio', ''),
                  ('txtCPFSocio', cpf),
                  ('txtBairroRapida', ''),
                  ('txtLogradouroRapida', ''),
                  ('btnPesquisaRapida', 'Pesquisar'),
                 ]
    #
    headers_parameters = {
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding":"gzip, deflate, br",
        "Accept-Language":"en-US,en;q=0.9",
        "Cache-Control":"max-age=0",
        "Content-Length":"17927",
        "Content-Type":"application/x-www-form-urlencoded",
        "Origin":"https://iged.jucepe.pe.gov.br",
        "Referer":"https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx",
        "Sec-Ch-Ua":"'Chromium';v='116', 'Not)A;Brand';v='24', 'Google Chrome';v='116'",
        "Sec-Ch-Ua-Mobile":"?0",
        "Sec-Ch-Ua-Platform":"'macOS'",
        "Sec-Fetch-Dest":"document",
        "Sec-Fetch-Mode":"navigate",
        "Sec-Fetch-Site":"same-origin",
        "Sec-Fetch-User":"?1",
        "Upgrade-Insecure-Requests":"1",
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
        }
    #
    dados_jucepe = s.post(url, data=parametros, headers=headers_parameters)
    soup = BeautifulSoup(dados_jucepe.content, 'lxml')
    try:
        # TENTAR
        table_dados = soup.find("table", attrs={"id":"tblEmpresas"})
    except:
        # SE DER ERRADO, CONTINUAR
        print("Nenhuma Empresa para este CPF")
        pass
    else:
        # FAZER ALGO
        for i,row in enumerate(table_dados.find_all('tr')):
            fields = {}
            cols = row.find_all('td')
            try:
                fields["nire"] = cols[0].text
                fields["nome"] = cols[1].text
                fields["natureza"] = cols[2].text
                fields["cnpj"] = cols[3].text
                fields["situacao"] = cols[4].text
                fields["municipio"] = cols[5].text
            except:
                print("ERRO EM LINHA!!")
                pass
            else:
                dados_empresas.append(fields)

ERRO EM LINHA!!


In [91]:
dados_empresas

[{'nire': '26200575378',
  'nome': '\n\n AGRO FACTORING FOMENTO COMERCIAL LTDA \n',
  'natureza': 'SOCIEDADE EMPRESÃ\x81RIA LIMITADA',
  'cnpj': '24.407.504.0001/99',
  'situacao': 'CANCELADA - ART.60 LEI 8934/94',
  'municipio': 'RECIFE'},
 {'nire': '26300003139',
  'nome': '\n\n ALFANOR S/A AGRICULTURA INDUSTRIA E COMERCIO DO NORDESTE \n',
  'natureza': 'SOCIEDADE ANÃ\x94NIMA FECHADA',
  'cnpj': '13.637.335.0001/67',
  'situacao': 'EXTINTA',
  'municipio': 'RECIFE'},
 {'nire': '26202114211',
  'nome': '\n\n ARACAJU LOCADORA DE EQUIPAMENTOS PARA COGERAÃ\x87Ã\x83O LTDA \n',
  'natureza': 'SOCIEDADE EMPRESÃ\x81RIA LIMITADA',
  'cnpj': '07.709.929.0001/33',
  'situacao': 'EXTINTA',
  'municipio': 'RECIFE'},
 {'nire': '26201156026',
  'nome': '\n\n AVICOLA DAGEMA LTDA \n',
  'natureza': 'SOCIEDADE EMPRESÃ\x81RIA LIMITADA',
  'cnpj': '.../',
  'situacao': 'CANCELADA - ART.60 LEI 8934/94',
  'municipio': 'RECIFE'},
 {'nire': '26300002132',
  'nome': '\n\n AVICOLA DAGEMA S/A \n',
  'natureza

In [92]:
import pandas as pd

In [93]:
df = pd.DataFrame(dados_empresas)

In [94]:
df

,nire,nome,natureza,cnpj,situacao,municipio
0,26200575378,\n\n AGRO FACTORING FOMENTO COMERCIAL LTDA \n,SOCIEDADE EMPRESÃRIA LIMITADA,24.407.504.0001/99,CANCELADA - ART.60 LEI 8934/94,RECIFE
1,26300003139,\n\n ALFANOR S/A AGRICULTURA INDUSTRIA E COMER...,SOCIEDADE ANÃNIMA FECHADA,13.637.335.0001/67,EXTINTA,RECIFE
2,26202114211,\n\n ARACAJU LOCADORA DE EQUIPAMENTOS PARA COG...,SOCIEDADE EMPRESÃRIA LIMITADA,07.709.929.0001/33,EXTINTA,RECIFE
3,26201156026,\n\n AVICOLA DAGEMA LTDA \n,SOCIEDADE EMPRESÃRIA LIMITADA,.../,CANCELADA - ART.60 LEI 8934/94,RECIFE
4,26300002132,\n\n AVICOLA DAGEMA S/A \n,SOCIEDADE ANÃNIMA FECHADA,09.357.161.0001/01,EXTINTA,RECIFE
5,26200529422,\n\n BOMPRECO COMERCIAL DE ALIMENTOS LTDA \n,SOCIEDADE EMPRESÃRIA LIMITADA,24.084.204.0001/16,CANCELADA - ART.60 LEI 8934/94,RECIFE
6,26300008173,\n\n BOMPRECO PARTICIPACOES S/A \n,SOCIEDADE ANÃNIMA FECHADA,35.597.038.0001/78,CANCELADA - ART.60 LEI 8934/94,RECIFE
7,26300009927,\n\n BOMPRECO TRUST DE RECEBIVEIS S/A \n,SOCIEDADE ANÃNIMA FECHADA,01.269.202.0001/04,CANCELADA - ART.60 LEI 8934/94,RECIFE
8,26200580916,\n\n BOMSERV FACTORING FOMENTO COMERCIAL LTDA \n,SOCIEDADE EMPRESÃRIA LIMITADA,24.458.077.0001/78,EXTINTA,RECIFE
9,26300010160,\n\n COMPANHIA BOMPRECO DE PARTICIPACOES E EMP...,SOCIEDADE ANÃNIMA FECHADA,.../,EXTINTA,RECIFE


In [95]:
cpfs = ["002.301.204/82","337.006.804/49"]

In [96]:
for cpf in cpfs:
    print(cpf)

002.301.204/82
337.006.804/49


In [97]:
url = "https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx"

dados_empresas = []

with requests.Session() as s:
    for cpf in cpfs:
        resultado = s.get(url)
        t = resultado.text
        soup_main = BeautifulSoup(t, 'lxml')
        result = soup_main.findAll('input', attrs={'name' : '__VIEWSTATE'})[0]
        VIEWSTATE =  result['value'];
        result = soup_main.findAll('input', attrs={'name' : '__VIEWSTATEGENERATOR'})[0]
        VIEWSTATEGENERATOR =  result['value']
        result = soup_main.findAll('input', attrs={'name' : '__EVENTVALIDATION'})[0]
        EVENTVALIDATION =  result['value']
        #
        parametros = [('__EVENTTARGET', ''), 
                      ('__EVENTARGUMENT', ''),
                      ('__VIEWSTATE', VIEWSTATE),
                      ('__VIEWSTATEGENERATOR', VIEWSTATEGENERATOR),
                      ('__EVENTVALIDATION', EVENTVALIDATION),
                      ('txtNire', ''),
                      ('txtCNPJ', ''),
                      ('txtNomePessoa', ''),
                      ('txtNomeSocio', ''),
                      ('txtCPFSocio', cpf),
                      ('txtBairroRapida', ''),
                      ('txtLogradouroRapida', ''),
                      ('btnPesquisaRapida', 'Pesquisar'),
                     ]
        #
        headers_parameters = {
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding":"gzip, deflate, br",
            "Accept-Language":"en-US,en;q=0.9",
            "Cache-Control":"max-age=0",
            "Content-Length":"17927",
            "Content-Type":"application/x-www-form-urlencoded",
            "Origin":"https://iged.jucepe.pe.gov.br",
            "Referer":"https://iged.jucepe.pe.gov.br/consultaempresa/Default.aspx",
            "Sec-Ch-Ua":"'Chromium';v='116', 'Not)A;Brand';v='24', 'Google Chrome';v='116'",
            "Sec-Ch-Ua-Mobile":"?0",
            "Sec-Ch-Ua-Platform":"'macOS'",
            "Sec-Fetch-Dest":"document",
            "Sec-Fetch-Mode":"navigate",
            "Sec-Fetch-Site":"same-origin",
            "Sec-Fetch-User":"?1",
            "Upgrade-Insecure-Requests":"1",
            "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
            }
        #
        dados_jucepe = s.post(url, data=parametros, headers=headers_parameters)
        soup = BeautifulSoup(dados_jucepe.content, 'lxml')
        try:
            # TENTAR
            table_dados = soup.find("table", attrs={"id":"tblEmpresas"})
        except:
            # SE DER ERRADO, CONTINUAR
            print("Nenhuma Empresa para este CPF")
            pass
        else:
            # FAZER ALGO
            for i,row in enumerate(table_dados.find_all('tr')):
                fields = {}
                cols = row.find_all('td')
                try:
                    fields["CPF"] = cpf                 ## ATENÇÃO LINHA NOVA AQUI
                    fields["nire"] = cols[0].text
                    fields["nome"] = cols[1].text
                    fields["natureza"] = cols[2].text
                    fields["cnpj"] = cols[3].text
                    fields["situacao"] = cols[4].text
                    fields["municipio"] = cols[5].text
                except:
                    print("ERRO EM LINHA!!")
                    pass
                else:
                    dados_empresas.append(fields)

ERRO EM LINHA!!
ERRO EM LINHA!!


In [98]:
df = pd.DataFrame(dados_empresas)

In [99]:
df

,CPF,nire,nome,natureza,cnpj,situacao,municipio
0,002.301.204/82,26200575378,\n\n AGRO FACTORING FOMENTO COMERCIAL LTDA \n,SOCIEDADE EMPRESÃRIA LIMITADA,24.407.504.0001/99,CANCELADA - ART.60 LEI 8934/94,RECIFE
1,002.301.204/82,26300003139,\n\n ALFANOR S/A AGRICULTURA INDUSTRIA E COMER...,SOCIEDADE ANÃNIMA FECHADA,13.637.335.0001/67,EXTINTA,RECIFE
2,002.301.204/82,26202114211,\n\n ARACAJU LOCADORA DE EQUIPAMENTOS PARA COG...,SOCIEDADE EMPRESÃRIA LIMITADA,07.709.929.0001/33,EXTINTA,RECIFE
3,002.301.204/82,26201156026,\n\n AVICOLA DAGEMA LTDA \n,SOCIEDADE EMPRESÃRIA LIMITADA,.../,CANCELADA - ART.60 LEI 8934/94,RECIFE
4,002.301.204/82,26300002132,\n\n AVICOLA DAGEMA S/A \n,SOCIEDADE ANÃNIMA FECHADA,09.357.161.0001/01,EXTINTA,RECIFE
5,002.301.204/82,26200529422,\n\n BOMPRECO COMERCIAL DE ALIMENTOS LTDA \n,SOCIEDADE EMPRESÃRIA LIMITADA,24.084.204.0001/16,CANCELADA - ART.60 LEI 8934/94,RECIFE
6,002.301.204/82,26300008173,\n\n BOMPRECO PARTICIPACOES S/A \n,SOCIEDADE ANÃNIMA FECHADA,35.597.038.0001/78,CANCELADA - ART.60 LEI 8934/94,RECIFE
7,002.301.204/82,26300009927,\n\n BOMPRECO TRUST DE RECEBIVEIS S/A \n,SOCIEDADE ANÃNIMA FECHADA,01.269.202.0001/04,CANCELADA - ART.60 LEI 8934/94,RECIFE
8,002.301.204/82,26200580916,\n\n BOMSERV FACTORING FOMENTO COMERCIAL LTDA \n,SOCIEDADE EMPRESÃRIA LIMITADA,24.458.077.0001/78,EXTINTA,RECIFE
9,002.301.204/82,26300010160,\n\n COMPANHIA BOMPRECO DE PARTICIPACOES E EMP...,SOCIEDADE ANÃNIMA FECHADA,.../,EXTINTA,RECIFE
